<a href="https://colab.research.google.com/github/ricardobizerra/data-analysis/blob/main/predicting_titanic's_survivors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [198]:
import pandas as pd
import numpy as np
import torch

In [199]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [200]:
# columns that are not relevant for the analysis are removed
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis='columns', inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [201]:
df.size

7128

In [202]:
# people that were 35
df[df['Age'] == 35]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
20,0,2,male,35.0,0,0,26.0000,S
211,1,2,female,35.0,0,0,21.0000,S
230,1,1,female,35.0,1,0,83.4750,S
258,1,1,female,35.0,0,0,512.3292,C
269,1,1,female,35.0,0,0,135.6333,S
279,1,3,female,35.0,1,1,20.2500,S
363,0,3,male,35.0,0,0,7.0500,S
383,1,1,female,35.0,1,0,52.0000,S


In [203]:
# removing empty or NaN data
df.dropna(inplace=True)
df = df[df['Fare'] != 0]
df.size

5640

In [204]:
df['AgeNorm'] = np.log10(df['Age'])
df['FareNorm'] = np.log10(df['Fare'])
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeNorm,FareNorm
0,0,3,male,22.0,1,0,7.2500,S,1.342423,0.860338
1,1,1,female,38.0,1,0,71.2833,C,1.579784,1.852988
2,1,3,female,26.0,0,0,7.9250,S,1.414973,0.898999
3,1,1,female,35.0,1,0,53.1000,S,1.544068,1.725095
4,0,3,male,35.0,0,0,8.0500,S,1.544068,0.905796
...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q,1.591065,1.464266
886,0,2,male,27.0,0,0,13.0000,S,1.431364,1.113943
887,1,1,female,19.0,0,0,30.0000,S,1.278754,1.477121
889,1,1,male,26.0,0,0,30.0000,C,1.414973,1.477121


In [205]:
# data normalization, in order to train a model
df['FirstClass'] = np.where(df['Pclass'] == 1, 1, 0)
df['SecondClass'] = np.where(df['Pclass'] == 2, 1, 0)

df['EmbarkedC'] = np.where(df['Embarked'] == 'C', 1, 0)
df['EmbarkedQ'] = np.where(df['Embarked'] == 'Q', 1, 0)

df['SexNorm'] = np.where(df['Sex'] == 'female', 1, 0)

df['Constant'] = 1

df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeNorm,FareNorm,FirstClass,SecondClass,EmbarkedC,EmbarkedQ,SexNorm,Constant
0,0,3,male,22.0,1,0,7.2500,S,1.342423,0.860338,0,0,0,0,0,1
1,1,1,female,38.0,1,0,71.2833,C,1.579784,1.852988,1,0,1,0,1,1
2,1,3,female,26.0,0,0,7.9250,S,1.414973,0.898999,0,0,0,0,1,1
3,1,1,female,35.0,1,0,53.1000,S,1.544068,1.725095,1,0,0,0,1,1
4,0,3,male,35.0,0,0,8.0500,S,1.544068,0.905796,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,5,29.1250,Q,1.591065,1.464266,0,0,0,1,1,1
886,0,2,male,27.0,0,0,13.0000,S,1.431364,1.113943,0,1,0,0,0,1
887,1,1,female,19.0,0,0,30.0000,S,1.278754,1.477121,1,0,0,0,1,1
889,1,1,male,26.0,0,0,30.0000,C,1.414973,1.477121,1,0,1,0,0,1


In [206]:
coeff = torch.randn(9).double()
coeff

tensor([ 1.1956, -0.1230, -1.6120,  0.6522, -0.2723, -0.6134, -0.9186, -2.3040,
        -1.6382], dtype=torch.float64)

In [207]:
data = torch.tensor(df[['AgeNorm','SibSp','Parch','FareNorm','FirstClass','SecondClass','EmbarkedC','EmbarkedQ', 'Constant']].values)
data

tensor([[1.3424, 1.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [1.5798, 1.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [1.4150, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        ...,
        [1.2788, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 1.0000],
        [1.4150, 0.0000, 0.0000,  ..., 1.0000, 0.0000, 1.0000],
        [1.5051, 0.0000, 0.0000,  ..., 0.0000, 1.0000, 1.0000]],
       dtype=torch.float64)

In [208]:
(data * coeff)[0].sum()

tensor(0.4050, dtype=torch.float64)

In [209]:
preds = data @ coeff.T
data[preds.isinf()]

tensor([], size=(0, 9), dtype=torch.float64)

In [210]:
labels = torch.tensor(df['Survived'].values)
labels.shape

torch.Size([705])

In [211]:
loss = ((preds-labels)**2).sum()
loss

tensor(2559.1305, dtype=torch.float64)

In [212]:
coeff = torch.randn(9).double().requires_grad_()
coeff

tensor([-0.0142, -2.0493, -1.2242,  1.3963, -0.5657, -0.1943, -0.2411, -0.1689,
         1.4577], dtype=torch.float64, requires_grad=True)

In [213]:
preds = data @ coeff.T
loss = ((preds-labels)**2).sum()
loss.backward()

In [214]:
coeff.grad

with torch.no_grad():
  coeff -= coeff.grad * 0.0001
  coeff.grad.zero_()

In [215]:
# Using Gradient Descent
for i in range(10):
  preds = data @ coeff.T
  labels = torch.tensor(df['Survived'].values)
  loss = ((preds-labels)**2).sum()
  loss.backward(retain_graph=True)

  with torch.no_grad():
    coeff -= coeff.grad * 0.0001
    coeff.grad.zero_()

In [216]:
labels

tensor([0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
        0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0,
        0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0,

In [217]:
preds

tensor([ 1.6122e-01,  1.4110e-01,  8.4363e-01,  3.1351e-01,  7.4296e-01,
         8.4421e-01, -1.7289e-01,  3.4502e-01,  4.2249e-01,  8.6196e-01,
         4.9344e-01,  9.4678e-01, -9.6263e-01,  1.0647e+00,  6.0347e-01,
        -8.8449e-01,  4.7737e-01,  1.0036e+00,  6.7794e-01,  8.8188e-01,
         8.9958e-01, -6.7781e-01, -9.5142e-01, -7.3786e-01,  3.6442e-01,
         3.3277e-01,  3.2127e-01,  2.3695e-01,  9.2901e-01, -2.4713e-02,
         2.5330e-01,  7.3294e-02,  2.9442e-01,  4.9098e-01,  7.8663e-01,
         6.6995e-01, -1.0223e+00,  9.1371e-01,  8.4213e-02,  3.7198e-01,
         2.5297e-01,  7.4985e-01,  4.8034e-01,  3.9409e-01, -2.1308e+00,
         5.7462e-01,  4.4138e-01, -6.1415e-01,  6.3229e-01,  9.7194e-01,
        -2.4517e+00, -5.1337e-01,  5.9644e-01, -2.2673e+00,  1.6936e+00,
         1.4974e-01,  1.7206e+00,  8.4079e-01,  1.7696e+00,  1.0116e+00,
         9.6617e-01,  8.9178e-01,  1.0367e+00,  8.2681e-01, -1.0073e+00,
         5.7452e-02, -8.0745e-01,  8.8037e-01,  8.1

Neural Nets

In [218]:
coeff = torch.randn((2,9)).double().requires_grad_()
coeff

tensor([[-1.0790,  1.5748,  2.2381,  0.2108, -0.1735, -0.0940, -0.1478,  2.3753,
         -1.7634],
        [ 0.8903, -1.3039, -0.1716,  2.2860,  1.5608,  1.0044,  1.3041,  0.3298,
          0.3781]], dtype=torch.float64, requires_grad=True)

In [219]:
preds = data @ coeff.T

In [220]:
preds

tensor([[-1.4558,  2.2361],
        [-1.8239,  7.5815],
        [-3.1007,  3.6930],
        ...,
        [-3.0053,  6.4541],
        [-3.3000,  7.8794],
        [-0.8247,  4.0808]], dtype=torch.float64, grad_fn=<MmBackward0>)

In [221]:
relu = torch.nn.ReLU()
preds = relu(preds)
preds

tensor([[0.0000, 2.2361],
        [0.0000, 7.5815],
        [0.0000, 3.6930],
        ...,
        [0.0000, 6.4541],
        [0.0000, 7.8794],
        [0.0000, 4.0808]], dtype=torch.float64, grad_fn=<ReluBackward0>)

In [222]:
preds = preds[:,0] + preds[:,1]
preds

tensor([ 2.2361,  7.5815,  3.6930,  5.9532,  3.8234,  7.4014,  5.1533,  5.0909,
         5.7821,  3.8914,  6.7643,  3.6071, 12.6514,  3.4447,  5.6845,  9.1330,
         3.2715,  5.9918,  5.2924,  3.8230,  6.7711,  4.6286, 12.6574, 10.7485,
         7.9675,  5.3369,  7.6045,  6.0030,  3.6259,  2.0537,  3.8008,  2.7330,
         4.3755,  9.0373,  3.8957,  3.0502,  6.1989,  3.5946,  7.7529,  4.6152,
         8.7825,  4.8941,  4.9413,  7.4007,  9.8153,  4.8412,  6.4995,  7.0923,
         5.0189,  3.6005,  7.8738,  1.2305,  5.0569,  9.6397,  6.8260,  4.2897,
         5.7233,  3.6427,  7.3248,  4.1987,  3.7547,  3.9152,  7.0518,  4.8124,
         2.1369,  8.6937, 10.7329,  3.6776,  3.7507,  3.5826,  6.1995,  6.0308,
         3.9214,  8.4110,  8.4027,  5.6873,  4.6767,  3.7180,  7.2607,  3.8725,
         1.2216,  3.7180,  3.5753,  3.8360,  7.3504,  4.0640,  3.6439,  2.5010,
         5.4297,  3.6104,  4.3863,  4.4031,  9.7720,  9.0023,  4.4770,  6.1077,
         5.2750,  7.6259,  3.7412,  3.55

In [223]:
loss = ((preds - labels)**2).sum()
loss

tensor(21989.2122, dtype=torch.float64, grad_fn=<SumBackward0>)

In [224]:
loss.backward()

In [225]:
coeff.grad

tensor([[2539.5181, 3515.5603, 4276.5742, 3561.2553,  594.2235,  503.4063,
          377.6676,  154.0788, 2209.2983],
        [9866.9004, 2731.8492, 3989.7867, 9773.6965, 2581.5952, 1744.7987,
         1736.3937,  228.1804, 6883.1942]], dtype=torch.float64)

In [226]:
with torch.no_grad():
    coeff -= coeff.grad * 0.001
    coeff.grad.zero_()

In [227]:
preds = relu(data @ coeff.T)
preds = preds[:,0] + preds[:,1]
loss = ((preds - labels)**2).sum()

In [228]:
epochs = 100000
learning_rate = 0.0001
coeff = torch.randn((2,9)).double().requires_grad_()
for i in range(epochs):
  preds = relu(data @ coeff.T)
  preds = preds[:,0] + preds[:,1]
  loss = ((preds - labels)**2).sum()
  loss.backward()
  print(loss)
  with torch.no_grad():
    coeff -= coeff.grad * learning_rate
    coeff.grad.zero_()

Streaming output truncated to the last 5000 lines.
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=torch.float64, grad_fn=<SumBackward0>)
tensor(287., dtype=to